In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
import scipy.sparse.linalg
from scipy.io import loadmat
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
def updateWLRAgrad(X, W, v, u, step=None, gamma=3, beta=4, reg_lambda=0.1):
    """
    Updates a single factor matrix (u) while keeping the other (v) fixed.
    Includes L2 regularization.
    """
    R = u.dot(v.T) - X  # Residual matrix
    WR = W.multiply(R)  # Masked residual
    gradu = WR.dot(v) + reg_lambda * u  # Gradient with L2 regularization

    if step is None or not step:  # Check if step is empty or None
        step = 2 * np.linalg.norm(u) / np.linalg.norm(gradu)

    e0 = np.sum(WR.power(2)) + reg_lambda * np.sum(u ** 2)
    e1 = float('inf')
    while e1 > e0:
        unew = u - step * gradu
        step /= gamma
        e1 = np.sum((W.multiply(X - unew.dot(v.T))).power(2)) + reg_lambda * np.sum(unew ** 2)
    u = unew
    step *= beta
    return u, step, e1

In [ ]:
# Load data matrix
data = loadmat('Data/matlab/inputX.mat')  # Load the .mat file
X = data['X']  # Assuming the matrix is stored under the key 'X'
X = csr_matrix(X)  # Convert to sparse matrix
m, n = X.shape
numX = X.count_nonzero()

# Construct binary weight matrix
W = X.copy()
W.data = np.ones_like(W.data)

rank = 5

# Init.2: average per row
v = np.ones((n, rank))
u = np.random.rand(m, rank)
for i in range(m):
    non_zero_indices = X[i].nonzero()[1]
    if len(non_zero_indices) > 0:
        u[i] = np.array([X[i, non_zero_indices].mean() for _ in range(len(u[i]))])

In [ ]:
# Estimate the parameters of the model: Optimize u and v alternatively via gradient descent
k = 1
max_k = 500
uprec = u
stepu = []
stepv = []
e = []
while k <= max_k and (k <= 2 or (e[-2] - e[-1]) / e[-2] > 1e-6):  # Stopping criterion
    # Update u
    u, stepu, eu = updateWLRAgrad(X, W, v, u, stepu)
    # Update v
    v, stepv, ev = updateWLRAgrad(X.T, W.T, u, v, stepv)
    e.extend([np.sqrt(eu / numX), np.sqrt(ev / numX)])
    print(f"Iteration {k}, RMSE: {e[-1]}")
    k += 1

# Display the evolution of the RMSE
plt.figure()
plt.xlabel('Iteration')
plt.ylabel('Observed RMSE')
plt.plot(e, 'bo--')
plt.show()

In [ ]:
# Construct the prediction in a csv file
data_eval = loadmat('Data/matlab/inputEval.mat')
Eval = data_eval['Eval']
numpred = Eval.shape[0]
predic = np.zeros((numpred,2))
# predic[:,0] = np.linspace(1, numpred, numpred, endpoint=True).round().astype(int)
for i in range(numpred):
    predic[i,0] = int(i + 1)
    predic[i,1] = np.dot(u[Eval[i,0]-1], v[Eval[i,1]-1])
    # Make sure the prediction is in [1,5]
    predic[i,1] = max(1, predic[i,1])
    predic[i,1] = min(5, predic[i,1])
np.savetxt('l2_reg.csv', predic, delimiter=',', fmt=["%d"] + ["%.4f"], header="ID,Rating",comments='')